In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

label_methods = [
    "random",
    "trend",
    "trend_profit",
    "trend_multi",
    "clusters",
    "multi_window",
    "validated_levels",
    "zigzag",
    "mean_rev",
    "mean_rev_multi",
    "mean_rev_vol",
    "filter",
    "multi_filter",
    "filter_bidirectional",
    "filter_one",
    "trend_one",
    "filter_flat",
]
search_methods = [
    'clusters',
    'markov',
    'lgmm',
    'wkmeans',
    'mapie',
    'causal',
]

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2020,1,1), train_end=datetime(2025,1,1),
        test_start=datetime(2022,1,1),  test_end=datetime(2023,6,1),
        label_method='fractal', search_type='lgmm', search_subtype='',
        n_models=1, debug=False,
    ),
]

# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['label_method']}_{cfg['search_type']}_{cfg.get('search_subtype', '')}".strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None
if not study:
    n_trials = 5000
else:
    n_trials = len(study.trials) + 5000

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 1/5000 score=0.534913 avg=27.96s mem=461.79MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 2/5000 score=0.566907 avg=20.74s mem=435.20MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 3/5000 score=0.591851 avg=25.07s mem=462.14MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 4/5000 score=0.591851 avg=20.12s mem=460.18MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 5/5000 score=0.591851 avg=18.05s mem=514.29MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 6/5000 score=0.591851 avg=15.89s mem=552.22MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 7/5000 score=0.591851 avg=14.69s mem=564.53MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 8/5000 score=0.591851 avg=15.56s mem=585.11MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 9/5000 score=0.591851 avg=15.46s mem=559.98MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 10/5000 score=0.591851 avg=15.10s mem=594.97MB 
[XAUUSD_H1_buy_fractal_lgmm] modelo 0 trial 11/5000 score=0.591851 avg=15.13s mem=567.80M